In [1]:
import pandas as pd
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")
train1 = train.drop(['Descript', 'Resolution'], axis = 1)

In [2]:
train1 = train1.query('Y != 90')

In [3]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
def to_timezone(hour_num):
    timezone = 8
    if 3<=hour_num<6:
        timezone = 1
    elif 6<=hour_num<9:
        timezone = 2
    elif 9<=hour_num<12:
        timezone = 3
    elif 12<=hour_num<15:
        timezone = 4
    elif 15<=hour_num<18:
        timezone = 5
    elif 18<=hour_num<21:
        timezone = 6
    elif 21<=hour_num<24:
        timezone = 7
    return timezone
train1['Timezone'] =  Dt.dt.hour.apply(to_timezone)
train1.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

train1['Category'] = le.fit_transform(train1['Category'])

train1.head()
# DayOfWeek...label,PdDistrict...count

,Category,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,Timezone
0,37,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,7
1,21,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,7
2,21,6,105284,0,-122.424363,37.800414,2015-05-13,2015,5,13,23,7
3,16,6,105284,1,-122.426995,37.800873,2015-05-13,2015,5,13,23,7
4,16,6,49311,1,-122.438738,37.771541,2015-05-13,2015,5,13,23,7


In [4]:
Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
def to_timezone(hour_num):
    timezone = 8
    if 3<=hour_num<6:
        timezone = 1
    elif 6<=hour_num<9:
        timezone = 2
    elif 9<=hour_num<12:
        timezone = 3
    elif 12<=hour_num<15:
        timezone = 4
    elif 15<=hour_num<18:
        timezone = 5
    elif 18<=hour_num<21:
        timezone = 6
    elif 21<=hour_num<24:
        timezone = 7
    return timezone
test['Timezone'] =  Dt.dt.hour.apply(to_timezone)
test.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
test['DayOfWeek'] = le.fit_transform(test['DayOfWeek'])

count_encoding = test['PdDistrict'].value_counts().to_dict()
test['PdDistrict'] = test['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)
test.head()

,Id,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour,Timezone
0,0,3,89591,1,-122.399588,37.735051,2015-05-10,2015,5,10,23,7
1,1,3,89591,0,-122.391523,37.732432,2015-05-10,2015,5,10,23,7
2,2,3,107017,1,-122.426002,37.792212,2015-05-10,2015,5,10,23,7
3,3,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,7
4,4,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,7


In [5]:
cols = ['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour','Timezone']
X = train1[cols]
Y = train1['Category']
X_test = test[cols]

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [6]:
from sklearn.ensemble import RandomForestClassifier as RFC

rf = RFC()

rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

In [7]:
accuracy_score(y_val, y_pred)

0.30075362098749026

In [8]:
xgb_submission = rf.predict(X_test)

In [9]:
import csv
pred = rf.predict_proba(test[['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour','Timezone']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)
submission.head()

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.0,0.203333,0.0,0.0,0.010000,0.0,0.00,0.000000,0.000,...,0.0,0.00,0.0,0.040000,0.0,0.00,0.040,0.200000,0.015000,0.042048
1,1,0.0,0.083333,0.0,0.0,0.000000,0.0,0.02,0.036667,0.000,...,0.0,0.01,0.0,0.000000,0.0,0.00,0.030,0.033333,0.153333,0.013333
2,2,0.0,0.140000,0.0,0.0,0.094286,0.0,0.00,0.000000,0.000,...,0.0,0.01,0.0,0.050000,0.0,0.01,0.070,0.090000,0.000000,0.000000
3,3,0.0,0.064000,0.0,0.0,0.000000,0.0,0.00,0.050000,0.015,...,0.0,0.01,0.0,0.043333,0.0,0.01,0.055,0.190000,0.045000,0.020000
4,4,0.0,0.064000,0.0,0.0,0.000000,0.0,0.00,0.050000,0.015,...,0.0,0.01,0.0,0.043333,0.0,0.01,0.055,0.190000,0.045000,0.020000


In [10]:
submission.to_csv('rf2-3.csv',index = False)